In [8]:
%pip install -q pandas demoji num2words tqdm nltk scikit-learn pandas transformers torch detoxify tensorflow keras 
%pip install keras-hub --no-deps

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import sys, os
sys.path.append(os.path.abspath('../'))

In [10]:
from pathlib import Path
from src.data_processing.loader import  load_dataset
from src.data_processing.preprocessing import preprocess_data


SAMPLE_LIMIT:int = 40000 
QUIET:bool = False
datasetPath:Path = Path("../data/dataset.txt")
preprocessed_path:Path = Path('../data/preprocessed.csv')

try:
    df = load_dataset(preprocessed_path, sample_limit=SAMPLE_LIMIT, quiet=QUIET, check_hash=False)
except FileNotFoundError:
    df = load_dataset(datasetPath, sample_limit=SAMPLE_LIMIT, quiet=QUIET, check_hash=True)
    df.to_csv(preprocessed_path) # При сохранении появялются Na
preprocess_data(df)
display(df.sample(10))

File ..\data\preprocessed.csv found. Loading...
No empty record found
Clrearing text...


100%|██████████| 40000/40000 [00:26<00:00, 1519.03it/s]


,text,normal,insult,threat,obscenity
134640,случилось,1.0,0.0,0.0,0.0
211733,это находится вообще карантин,1.0,0.0,0.0,0.0
116078,медали гонки,1.0,0.0,0.0,0.0
133859,красотища каких нет великолепная актриса женщина,1.0,0.0,0.0,0.0
4365,яб её обе дырочки отодрал,0.0,0.0,0.0,1.0
51124,это лебедь выпь,1.0,0.0,0.0,0.0
67353,пизды дать,0.0,0.0,1.0,0.0
65851,четвертым,1.0,0.0,0.0,0.0
85611,повесить яйца собаковода площадичто б другим п...,0.0,0.0,1.0,0.0
93404,вообще получаем12 с,1.0,0.0,0.0,0.0


In [11]:
def display_label_stats(df) -> None:
    display(df.loc[:, df.columns != "text"].value_counts().reset_index(name="count"))

display_label_stats(df)

,normal,insult,threat,obscenity,count
0,1.0,0.0,0.0,0.0,32731
1,0.0,1.0,0.0,0.0,4690
2,0.0,1.0,1.0,0.0,1041
3,0.0,0.0,1.0,0.0,866
4,0.0,0.0,0.0,1.0,347
5,0.0,1.0,0.0,1.0,280
6,0.0,1.0,1.0,1.0,37
7,0.0,0.0,1.0,1.0,8


In [12]:
import pandas as pd

val_samples = 150
val_df = df.sample(n=val_samples, replace=False, random_state=42)
df = pd.concat([df, val_df]).drop_duplicates(keep=False)
display(val_df)
val_texts = val_df.pop("text").to_list()
val_labels = val_df.to_numpy()

,text,normal,insult,threat,obscenity
5127,вставай страна огромная вставай смертный бой,1.0,0.0,0.0,0.0
32494,талантливый актер светлая память этому великом...,1.0,0.0,0.0,0.0
36340,аминь аминь аминь,1.0,0.0,0.0,0.0
43171,кроме очень очень денег помощи другим странам ...,1.0,0.0,0.0,0.0
166538,художник мастер картины живые сразу вспоминает...,1.0,0.0,0.0,0.0
...,...,...,...,...,...
72819,выглядит конек горбунок,1.0,0.0,0.0,0.0
25683,б сее оргаию моссипск ребят е это пиздец аsоvа...,0.0,1.0,0.0,0.0
192004,остался коврик,1.0,0.0,0.0,0.0
228631,классный петушок пригрел курочку,1.0,0.0,0.0,0.0


In [13]:
texts = df.pop("text").to_list()
labels = df.to_numpy()
texts

['старая немного трансформированная история советское время власть хотела сделать какуюто хрень объявляла всё делается просьбам трудящихся',
 'это сваю собаку нашли',
 'чурки пидорасы',
 'почему рухнул советский союз верхушка кпсс жила совести 19 000 000 членов кпсс мог ещё гордиться партией ум честь совесть нашей эпохи слова дела очень круто стали давно расходиться собой развалив союз детки кпсс успокоились продолжили валить партию россию посмотрите сегодняшний российский общак выкормыши бывшей кпсс делалось подтишка подполы стало законом чему пришли единожды солгавший тебе поверит думаю компартии будущего идея шикарная среднего ума мир изменился сильно нужны новые люди новые дороги которым пойдём светлое будущее',
 'поняли все что болел районе рассказывают пишут диагноз пневмония лечат коронавируса симптомы коронавируса держат взаперти инфекционных отделениях заходят больным называемой пневмонией защитных костюмах пишут больные записку врачам обслуживающему персоналу стекло показываю

In [14]:
from src.data_processing.encoding import TextVectorizer

encoder = TextVectorizer(texts)

In [15]:
from src.models import hate_model


model = hate_model.ToxicClassificationModel(encoder, encoder.input_length, encoder.max_tokens)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 26, 128)   │ 11,355,648 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 26)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     98,816 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │        260 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,462,980 (43.73 MB)

 Trainable params: 11,462,980 (43.73 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.train(texts, labels, batch_size=32, epochs=100, patience=10)

Epoch 1/100
971/971 - 77s - 79ms/step - accuracy: 0.8871 - auc: 0.9642 - loss: 0.2030 - val_accuracy: 0.9174 - val_auc: 0.9836 - val_loss: 0.1358 - learning_rate: 1.0000e-03
Epoch 2/100
971/971 - 73s - 75ms/step - accuracy: 0.9605 - auc: 0.9962 - loss: 0.0610 - val_accuracy: 0.9087 - val_auc: 0.9803 - val_loss: 0.1463 - learning_rate: 1.0000e-03
Epoch 3/100
971/971 - 75s - 78ms/step - accuracy: 0.9771 - auc: 0.9992 - loss: 0.0258 - val_accuracy: 0.9065 - val_auc: 0.9759 - val_loss: 0.1777 - learning_rate: 1.0000e-03
Epoch 4/100
971/971 - 76s - 78ms/step - accuracy: 0.9827 - auc: 0.9996 - loss: 0.0123 - val_accuracy: 0.9110 - val_auc: 0.9706 - val_loss: 0.2238 - learning_rate: 1.0000e-03
Epoch 5/100
971/971 - 75s - 78ms/step - accuracy: 0.9871 - auc: 0.9997 - loss: 0.0072 - val_accuracy: 0.9026 - val_auc: 0.9649 - val_loss: 0.2626 - learning_rate: 1.0000e-03
Epoch 6/100
971/971 - 75s - 78ms/step - accuracy: 0.9892 - auc: 0.9998 - loss: 0.0047 - val_accuracy: 0.9108 - val_auc: 0.9661 - v

In [17]:
model.save()

In [18]:
model.score(val_texts, val_labels.tolist())

Log Loss: 0.3203


c:\Users\ShapeCat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [19]:
labels_agregated = [model.agregate_proba(x) for x in val_labels]
model.score(val_texts, labels_agregated)

Log Loss: 0.1382
Accuracy: 0.9600


In [20]:
from src.models.detoxify import DetoxifyModel


model = DetoxifyModel()
model.score(val_texts, labels_agregated)

Log Loss: 0.2508
Accuracy: 0.9000


In [21]:
for name, module in model.model.model.named_modules():
    print(f"{name}: {type(module)}")

: <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>
roberta: <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaModel'>
roberta.embeddings: <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaEmbeddings'>
roberta.embeddings.word_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
roberta.embeddings.position_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
roberta.embeddings.token_type_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
roberta.embeddings.LayerNorm: <class 'torch.nn.modules.normalization.LayerNorm'>
roberta.embeddings.dropout: <class 'torch.nn.modules.dropout.Dropout'>
roberta.encoder: <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaEncoder'>
roberta.encoder.layer: <class 'torch.nn.modules.container.ModuleList'>
roberta.encoder.layer.0: <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaLayer'>
roberta.encoder.layer.0.attentio

In [22]:
from src.models.rubert import RuBertModel


model = RuBertModel()
model.score(val_texts, labels_agregated)

Log Loss: 0.3850
Accuracy: 0.8400


In [23]:
for name, module in model.model.named_modules():
    print(f"{name}: {type(module)}")

: <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
bert: <class 'transformers.models.bert.modeling_bert.BertModel'>
bert.embeddings: <class 'transformers.models.bert.modeling_bert.BertEmbeddings'>
bert.embeddings.word_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
bert.embeddings.position_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
bert.embeddings.token_type_embeddings: <class 'torch.nn.modules.sparse.Embedding'>
bert.embeddings.LayerNorm: <class 'torch.nn.modules.normalization.LayerNorm'>
bert.embeddings.dropout: <class 'torch.nn.modules.dropout.Dropout'>
bert.encoder: <class 'transformers.models.bert.modeling_bert.BertEncoder'>
bert.encoder.layer: <class 'torch.nn.modules.container.ModuleList'>
bert.encoder.layer.0: <class 'transformers.models.bert.modeling_bert.BertLayer'>
bert.encoder.layer.0.attention: <class 'transformers.models.bert.modeling_bert.BertAttention'>
bert.encoder.layer.0.attention.self: <class 'transformers.models.be